In [1]:
import numpy as np
from scipy.special import erfcinv

class EdgeComputingEnvironment:
    def __init__(self, M=5, area_size=100, D_m=1354, eta_m_range=(100, 300), F_max_ue=1.5e9, P_max=23, B=5e6, T_max=10e-3, F_max_es=30e9, S_max_es=60e3, epsilon=10**-7, E_max=3e-3, theta=10**-26, L=8, phi=1.0, N0_dbm=-174, simulation_max_time=1000):
        """
        Initialize the edge computing environment with given parameters.
        """
        self.M = M  # Number of users
        self.area_size = area_size  # Size of the area in which users are distributed
        self.D_m = D_m  # Task data size
        self.eta_m_range = eta_m_range  # Range of computation requirements
        self.F_max_ue = F_max_ue  # Maximum frequency of user equipment
        self.P_max = 10 ** (P_max / 10)  # Convert maximum transmission power from dB to Watts
        self.B = B  # Bandwidth
        self.T_max = T_max  # Maximum tolerable delay
        self.F_max_es = F_max_es  # Maximum frequency of edge server
        self.S_max_es = S_max_es  # Maximum storage size of edge server
        self.epsilon = epsilon  # Error tolerance for rate calculation
        self.E_max = E_max  # Maximum energy consumption
        self.theta = theta  # Energy coefficient
        self.L = L  # Number of antennas
        self.phi = phi  # Transmission probability
        self.R_min = 1e6  # Minimum data rate
        self.N0_dbm = N0_dbm  # Noise power in dBm
        self.N0 = 10 ** (N0_dbm / 10) / 1000  # Convert noise power from dBm/Hz to Watts/Hz
        self.simulation_max_time = simulation_max_time  # Maximum simulation time
        self.PL_d = lambda d: 10 ** ((-35.3 - 37.6 * np.log10(d)) / 10)  # Path loss model


        self.user_device_params = []  # List to store parameters for each user device
        self.initialize_user_device_params()  # Initialize user device parameters

        self.server_params = self.initialize_server_params()  # Initialize server parameters

        self.cache = []  # Cache to store tasks
        self.current_cache_size = 0  # Current size of the cache
        self.transmitting_tasks = []  # List to store transmitting tasks
        self.processing_tasks = []  # List to store processing tasks
        self.current_time = 0  # Current simulation time
        self.total_delay = 0  # Initialize total delay
        self.total_energy = 0  # Initialize total energy consumption

        # Initialize bandwidth and computation attributes
        self.total_bandwidth = 0 # Initialize total bandwidth
        self.total_computation = 0 # Initialize total computation
        self.avg_delay = 0
        self.avg_energy = 0
        # Metrics for averaging
        self.cumulative_delay = 0
        self.cumulative_energy = 0
        self.task_count = 0


    def initialize_user_device_params(self):
        """
        Initialize parameters for each user device.
        """
        for device_id in range(self.M):
            f_ue_m = np.random.choice(np.linspace(1, self.F_max_ue, 10, dtype=int))  # User device frequency
            f_ue_est = f_ue_m * 0.02  # Estimated frequency for user device
            p_m = np.random.choice(np.linspace(1, self.P_max, 10, dtype=int))  # Transmission power
            b_m = np.random.choice(np.linspace(1, self.B, 10, dtype=int))  # Bandwidth
            d = np.random.uniform(0, self.area_size / 2)  # Distance to server                                                        # distance to server must be 0 to (qotr moraba/2)
            g_m = np.array([self.PL_d(d)])  # Path loss
            h_bar = np.random.randn(1, self.L) + 1j * np.random.randn(1, self.L)  # Channel gain
            eta_m = np.random.choice(np.linspace(self.eta_m_range[0], self.eta_m_range[1], 10))  # Computation requirement

            self.user_device_params.append({
                'device_id': device_id,  # Assign a unique ID to each device
                'f_ue_m': f_ue_m,
                'f_ue_est': f_ue_est,
                'p_m': p_m,
                'b_m': b_m,
                'd': d,
                'g_m': g_m,
                'h_bar': h_bar,
                'eta_m': eta_m
            })

    def initialize_server_params(self):
        """
        Initialize parameters for the edge server.
        """
        f_es_m = np.random.choice(np.linspace(1, self.F_max_es, 10, dtype=int))  # Server frequency
        f_es_est = f_es_m * 0.02  # Estimated server frequency

        return {
            'f_es_m': f_es_m,
            'f_es_est': f_es_est,
            'S_max_es': self.S_max_es  # Maximum storage size
        }

    def create_task(self):
        """
        Create a new task for a specific user.
        """
        eta_m = np.random.choice(np.linspace(self.eta_m_range[0], self.eta_m_range[1], 10))  # Computation requirement
        T_max_task = np.random.choice(np.linspace(self.T_max / 2, self.T_max, 10))  # Maximum tolerable delay for task
        T_max_task = 0.001 # Static for article
        task_info = {
            'eta_m': eta_m,
            'T_max': T_max_task,
            'D_m': self.D_m  # Task data size
        }
        return task_info

    def calculate_gamma_m(self, b_m, p_m, user_id):
        """
        Calculate the signal-to-noise ratio (SNR) for a given user.
        """
        h_m = np.sqrt(self.user_device_params[user_id]['g_m'])[:, None] * self.user_device_params[user_id]['h_bar']  # Channel gain
        gamma_m = (p_m * np.linalg.norm(h_m, axis=1) ** 2) / (b_m * self.B * self.N0)  # SNR

        return gamma_m

    def calculate_uplink_rate(self, b_m, p_m, user_id):
        """
        Calculate the uplink data rate for a given user.
        """
        gamma_m = self.calculate_gamma_m(b_m, p_m, user_id)  # SNR
        V_m = 1 - (1 / (1 + gamma_m) ** 2)  # Intermediate variable for rate calculation
        Q_inv = np.sqrt(2) * erfcinv(2 * self.epsilon)  # Inverse Q-function
        R_m = (self.B / np.log(2)) * (b_m * np.log(1 + gamma_m) - np.sqrt((b_m * V_m) / (self.phi * self.B)) * Q_inv)  # Uplink data rate

        return R_m

    def calculate_delay(self, alpha_m, cache_hit, b_m, p_m, D_m, f_ue_m, f_es_m, f_ue_est, f_es_est, eta_m, user_id):
        """
        Calculate the end-to-end delay for a given task.
        """
        actual_f_ue_m = f_ue_m - f_ue_est  # Actual processing rate of the user device

        if cache_hit == 1:
            T_e2e = (1 - alpha_m) * eta_m * D_m / (f_es_m - f_es_est)  # Delay if task is in cache
        else:
            T_ue = (alpha_m * eta_m * D_m) / actual_f_ue_m  # User device processing delay
            R_m = self.calculate_uplink_rate(b_m, p_m, user_id)  # Uplink data rate
            T_tr = D_m / R_m  # Transmission delay
            T_es = (1 - alpha_m) * eta_m * D_m / (f_es_m - f_es_est)  # Edge server processing delay
            T_e2e = T_ue + T_tr + T_es  # Total end-to-end delay

        return T_e2e

    def calculate_energy_consumption(self, s_m, b_m, alpha_m, p_m, D_m, f_ue_m, f_ue_est, eta_m, user_id):
        """
        Calculate the energy consumption for a given task.
        """
        R_m = self.calculate_uplink_rate(b_m, p_m, user_id)  # Calculate uplink data rate

        actual_f_ue_m = f_ue_m - f_ue_est  # Calculate the actual processing rate of the UE

        E_ue = alpha_m * (self.theta / 2 * eta_m * D_m * (actual_f_ue_m ** 2))  # Energy consumption at the user device
        E_tx = ((1 - alpha_m) * D_m * p_m) / R_m  # Transmission energy

        if s_m == 1:  # Task is in cache
            E_total = 0  # No energy consumed when task is in cache
        else:
            E_total = (1 - s_m) * (E_ue + E_tx)  # Total energy consumption


        return E_total

    def manage_cache(self, task_info, task_delay):
        """
        Manage the cache for storing and retrieving tasks.
        """
        if task_delay == 0:
            for task in self.cache:
                if task_info == task[0]:  # Check if the task is already in cache
                    return True
            return False



        task_size = task_info['D_m']  # Task size
        Server_Max_Capacity = self.server_params['S_max_es']  # Server maximum capacity

        if (task_size + self.current_cache_size) <= Server_Max_Capacity:
            self.cache.append((task_info, task_delay))  # Add task to cache
            self.current_cache_size += task_size  # Update cache size
            return True
        else:
            sorted_cache = sorted(self.cache, key=lambda x: x[1], reverse=True)  # Sort tasks by delay in descending order

            while (task_size + self.current_cache_size) > Server_Max_Capacity:
                if not sorted_cache:
                    break  # Exit loop if sorted_cache is empty
                last_task = sorted_cache.pop()  # Remove the last task from sorted_cache
                self.cache.remove(last_task)  # Remove the task from the cache
                self.current_cache_size -= last_task[0]['D_m']  # Update current cache size

            self.cache.append((task_info, task_delay))  # Add task to cache
            self.current_cache_size += task_size  # Update cache size

            return True

    def reset_cumulative_metrics(self):
        self.cumulative_delay = 0
        self.cumulative_energy = 0
        self.task_count = 0

    def get_average_metrics(self):
        if self.task_count == 0:
            return {
                'average_delay': 0,
                'average_energy': 0
            }
        return {
            'average_delay': self.cumulative_delay / self.task_count,
            'average_energy': self.cumulative_energy / self.task_count
        }

    def step(self, actions):
        """
        Execute a single simulation step.
        """
        # Initialize cumulative metrics for the step
        task_rewards = []  # List to store reward for each task
        individual_task_params = []  # List to store individual task parameters
        num_tasks = len(actions)  # Number of tasks
        total_delay = 0  # Initialize total delay
        total_energy = 0  # Initialize total energy consumption

        for user_id, action in enumerate(actions):
            # Create a new task (user_id not necessary for task creation in this case)
            task = self.create_task()
            # Determine if the task is a cache hit or miss
            cache_hit = 1 if self.manage_cache(task, 0) else 0
            f_es_est = action['f_es_m'] * 0.02
            f_ue_est = action['f_ue_m'] * 0.02


            # Calculate the end-to-end delay for the task
            delay = self.calculate_delay(
                action['alpha_m'], cache_hit, action['b_m'], action['p_m'],
                task['D_m'], action['f_ue_m'], action['f_es_m'], f_ue_est,
                f_es_est, task['eta_m'], user_id
            )
            total_delay += delay
            self.cumulative_delay += delay
            self.total_delay += delay
            # Calculate the energy consumption for the task
            energy = self.calculate_energy_consumption(
                cache_hit, action['b_m'], action['alpha_m'], action['p_m'], task['D_m'], action['f_ue_m'],
                f_es_est, task['eta_m'], user_id
            )
            total_energy += energy
            self.cumulative_energy += energy
            self.total_energy += energy

            # Calculate the uplink data rate for the user
            R_m = self.calculate_uplink_rate(action['b_m'], action['p_m'], user_id)

            # Manage task transmission and processing times
            if cache_hit == 0:
                transmission_end_time = self.current_time + task['D_m'] / R_m
                processing_end_time = transmission_end_time + (1 - action['alpha_m']) * task['eta_m'] * task['D_m'] / (action['f_es_m'] - self.server_params['f_es_est'])

                self.transmitting_tasks.append((self.current_time, transmission_end_time, action['b_m']))
                self.processing_tasks.append((transmission_end_time, processing_end_time, action['f_es_m']))

                # Update cache with the task if it becomes eligible
                self.manage_cache(task, delay)
            else:
                # For cache hit, only processing delay is considered
                processing_end_time = self.current_time + (1 - action['alpha_m']) * task['eta_m'] * task['D_m'] / (action['f_es_m'] - self.server_params['f_es_est'])
                self.processing_tasks.append((self.current_time, processing_end_time, action['f_es_m']))

            # Store individual task parameters
            individual_task_params.append({
                'delay': delay,
                'alpha_m': action['alpha_m'],
                'task_reward': None,  # Will be calculated later
                'energy': energy,
                'R_m': R_m,
                'cache_hit': cache_hit,
                'device_id': user_id,
                'b_m': action['b_m'],
                'p_m': action['p_m'],
                'f_ue_m': action['f_ue_m'],
                'f_es_m': action['f_es_m'],
                'eta_m': task['eta_m'],
                'T_max': task['T_max']
            })


        # Calculate total bandwidth and computation resource usage at current time
        self.total_bandwidth = sum(b for _, end_time, b in self.transmitting_tasks if end_time > self.current_time)
        self.total_computation = sum(f for _, end_time, f in self.processing_tasks if end_time > self.current_time)

        # Free resources for tasks that have completed transmission or processing
        self.transmitting_tasks = [(start_time, end_time, b) for start_time, end_time, b in self.transmitting_tasks if end_time > self.current_time]
        self.processing_tasks = [(start_time, end_time, f) for start_time, end_time, f in self.processing_tasks if end_time > self.current_time]
        Penalties = []
        # Calculate reward
        for params in individual_task_params:
            task_reward = -params['delay'] - params['energy'] * 1e3

            # Apply penalties for exceeding resource limits
            if params['delay'] > params['T_max']:
                task_reward -= 1e6
                Penalties.append("T_max")
            if params['energy'] > self.E_max:
                task_reward -= 1e6
                Penalties.append("E_max")
            if params['R_m'] < self.R_min:
                task_reward -= 1e6
                Penalties.append("R_m")

            print(Penalties)

            params['task_reward'] = task_reward
            task_rewards.append(task_reward)


        # Calculate average metrics per task
        self.avg_delay = total_delay / num_tasks if num_tasks > 0 else 0
        self.avg_energy = total_energy / num_tasks if num_tasks > 0 else 0
        reward = sum(task_rewards) / num_tasks if num_tasks > 0 else 0

        if self.total_bandwidth > 1:
                reward -= 1e6
                Penalties.append("Bandwith")
        if self.total_computation > self.F_max_es:
                reward -= 1e6
                Penalties.append("F_Max")

        print(Penalties)

        # Increment current simulation time
        self.current_time += 1

        # Check if the simulation has reached its maximum allowed time
        done = self.current_time >= self.simulation_max_time

        # Check if the cumulative reward is below a certain threshold
        if reward < -1e5:
            done = True

        # Prepare the next state
        next_state = self.get_state()

        return reward, next_state, done

    def reset(self):
        """
        Reset the environment to its initial state.
        """
        self.cache = []  # Clear cache
        self.current_cache_size = 0  # Reset cache size
        self.transmitting_tasks = []  # Clear transmitting tasks
        self.processing_tasks = []  # Clear processing tasks
        self.current_time = 0  # Reset current time
        self.reset_cumulative_metrics()  # Reset cumulative metrics
        self.initialize_user_device_params()  # Reinitialize user device parameters
        self.avg_delay = 0
        self.avg_energy = 0
        self.server_params = self.initialize_server_params()  # Reinitialize server parameters

        initial_state = self.get_state()  # Get the initial state
        return initial_state

    def get_state(self):
        """
        Get the current state of the environment.
        """
        state = {
            'total_bandwidth': self.total_bandwidth,
            'total_computation': self.total_computation,
            'total_delay': self.total_delay,  # Include total delay in the state
            'total_energy': self.total_energy,  # Include total energy consumption in the state
            'current_time': self.current_time,
            'cache_size': self.current_cache_size,  # Include current cache size
            'transmitting_tasks': self.transmitting_tasks,  # transmitting tasks
            'processing_tasks': self.processing_tasks,  # processing tasks
        }
        return state

    def render(self):
        print(f"Total Bandwidth Used: {self.total_bandwidth}")
        print(f"Total Computation Used: {self.total_computation}")
        print(f"Current Cache Size: {self.current_cache_size}")
        print(f"Number of Transmitting Tasks: {len(self.transmitting_tasks)}")
        print(f"Number of Processing Tasks (Not Exist In Cache): {len(self.processing_tasks)}")
        print(f"Total Delay: {self.total_delay}")
        print(f"Total Energy Consumption: {self.total_energy}")
        print(f"avg_delay: {self.avg_delay}")
        print(f"avg_energy: {self.avg_energy}")



In [2]:
import numpy as np

def test_environment():
    # Initialize the environment
    env = EdgeComputingEnvironment()

    # Reset the environment to get the initial state
    state = env.reset()

    # Define the number of steps to test
    num_steps = 10

    for step in range(num_steps):
        print(f"\nStep {step + 1}")

        # Generate random actions for each user
        actions = []
        for user_id in range(env.M):
            action = {
                'alpha_m': np.random.uniform(0, 1),
                'b_m': np.random.uniform(0, 0.3),
                'p_m': np.random.choice(np.linspace(1, env.P_max, 10)),
                'f_ue_m': np.random.choice(np.linspace(1, env.F_max_ue, 10)),
                'f_es_m': np.random.choice(np.linspace(1, env.F_max_es/10, 10))
            }
            actions.append(action)

        # Perform a step in the environment with the generated actions
        reward, next_state, done = env.step(actions)

        # Print the parameters
        print("Actions:", actions)
        print("Reward:", reward)
        print("Next State:", next_state)
        print("Done:", done)

        # Render the environment's current state
        env.render()

        #if done:
        #    break

# Run the test
test_environment()



Step 1
['T_max', 'E_max', 'R_m']
['T_max', 'E_max', 'R_m']
['T_max', 'E_max', 'R_m']
['T_max', 'E_max', 'R_m', 'E_max']
['T_max', 'E_max', 'R_m', 'E_max']
['T_max', 'E_max', 'R_m', 'E_max']
Actions: [{'alpha_m': 0.9451283941282727, 'b_m': 0.002486675305642405, 'p_m': 45.11694033264175, 'f_ue_m': 333333334.1111111, 'f_es_m': 1333333333.8888888}, {'alpha_m': 0.520066163541077, 'b_m': 0.0998096025756637, 'p_m': 23.058470166320873, 'f_ue_m': 833333333.7777778, 'f_es_m': 1666666667.1111112}, {'alpha_m': 0.45744936963826655, 'b_m': 0.22131527642148, 'p_m': 45.11694033264175, 'f_ue_m': 1000000000.3333333, 'f_es_m': 2666666666.7777777}, {'alpha_m': 0.27827891469147925, 'b_m': 0.13261512043171017, 'p_m': 89.2338806652835, 'f_ue_m': 1000000000.3333333, 'f_es_m': 2000000000.3333333}, {'alpha_m': 0.5565687280959329, 'b_m': 0.20810911505737412, 'p_m': 23.058470166320873, 'f_ue_m': 1500000000.0, 'f_es_m': 1333333333.8888888}]
Reward: [-800003.16433398]
Next State: {'total_bandwidth': 0.456226674734

c:\Users\REZA\anaconda3\Lib\site-packages\numpy\core\function_base.py:182: RuntimeWarning: invalid value encountered in cast
  return y.astype(dtype, copy=False)
